In [1]:
#from https://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries
import scipy.io as spio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

In [2]:
import os

import mne
import mne_bids


In [3]:
def get_all_files(path, suffix, get_bids=False, prefix=None, bids_root=None, verbose=False, extension=None):
    """Return all files in all (sub-)directories of path with given suffixes and prefixes (case-insensitive).

    Args:
        path (string)
        suffix (iterable): e.g. ["vhdr", "edf"] or ".json"
        get_bids (boolean): True if BIDS_Path type should be returned instead of string. Default: False
        bids_root (string/path): Path of BIDS root folder. Only required if get_bids=True.
        prefix (iterable): e.g. ["SelfpacedRota", "ButtonPress] (optional)

    Returns:
        filepaths (list of strings or list of BIDS_Path)
    """

    if isinstance(suffix, str):
        suffix = [suffix]
    if isinstance(prefix, str):
        prefix = [prefix]

    filepaths = []
    for root, dirs, files in os.walk(path):
        for file in files:
            for suff in suffix:
                if file.endswith(suff.lower()):
                    if not prefix:
                        filepaths.append(os.path.join(root, file))
                    else:
                        for pref in prefix:
                            if pref.lower() in file.lower():
                                filepaths.append(os.path.join(root, file))

    bids_paths = filepaths
    if get_bids:
        if not bids_root:
            print("Warning: No root folder given. Please pass bids_root parameter to create a complete BIDS_Path object.")
        bids_paths = []
        for filepath in filepaths:
            entities = mne_bids.get_entities_from_fname(filepath)
            try:
                bids_path = mne_bids.BIDSPath(subject=entities["subject"], session=entities["session"], task=entities["task"], run=entities["run"], acquisition=entities["acquisition"], suffix=entities["suffix"], extension=extension, root=bids_root)
            except ValueError as err:
                print(f"ValueError while creating BIDS_Path object for file {filepath}: {err}")
            else:
                bids_paths.append(bids_path)

    if verbose:
        if not bids_paths:
            print("No corresponding files found.")
        else:    
            print('Corresponding files found:')
            for idx, file in enumerate(bids_paths):
                print(idx, ':', os.path.basename(file))
                
    return bids_paths

In [4]:
root = r'C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata'
electrodes = get_all_files(
    path=root,
    suffix='electrodes.tsv',
    get_bids=False,
    prefix=None,
    bids_root=root,
    verbose=True,
    extension='tsv')

Corresponding files found:
0 : sub-001_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv
1 : sub-001_ses-EphysMedOn01_space-MNI152NLin2009bAsym_electrodes.tsv
2 : sub-001_ses-EphysMedOn02_space-MNI152NLin2009bAsym_electrodes.tsv
3 : sub-002_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv
4 : sub-002_ses-EphysMedOff02_space-MNI152NLin2009bAsym_electrodes.tsv
5 : sub-002_ses-EphysMedOff03_space-MNI152NLin2009bAsym_electrodes.tsv
6 : sub-003_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv
7 : sub-003_ses-EphysMedOn01_space-MNI152NLin2009bAsym_electrodes.tsv
8 : sub-003_ses-EphysMedOn02_space-MNI152NLin2009bAsym_electrodes.tsv
9 : sub-003_ses-EphysMedOn03_space-MNI152NLin2009bAsym_electrodes.tsv
10 : sub-004_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv
11 : sub-004_ses-EphysMedOn01_space-MNI152NLin2009bAsym_electrodes.tsv
12 : sub-004_ses-EphysMedOn02_space-MNI152NLin2009bAsym_electrodes.tsv
13 : sub-005_ses-EphysMedOff01_space-MNI152NLin2009bAs

In [5]:
import pandas as pd

In [6]:
dfs = list()
for file in electrodes:
    df = pd.read_csv(file, sep='\t', index_col=0)
    dfs.append(df)

In [7]:
root = r'C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\sourcedata'
files = get_all_files(
    path=root,
    suffix='ea_reconstruction.mat',
    get_bids=False,
    prefix=None,
    bids_root=root,
    verbose=True,
    extension='None')

Corresponding files found:
0 : ea_reconstruction.mat
1 : ea_reconstruction.mat
2 : ea_reconstruction.mat
3 : ea_reconstruction.mat
4 : ea_reconstruction.mat


In [10]:
coords = list()
for file in files:
    print(file)
    coord = loadmat(file)
    coords.append(coord)

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\sourcedata\sub-001\Neuroimaging\ea_reconstruction.mat
C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\sourcedata\sub-002\Neuroimaging\ea_reconstruction.mat
C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\sourcedata\sub-003\Neuroimaging\ea_reconstruction.mat
C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\sourcedata\sub-004\Neuroimaging\ea_reconstruction.mat
C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\sourcedata\sub-005\Neuroimaging\ea_reconstruction.mat


# sub-001

In [11]:
coord = coords[0]

In [26]:
coord.keys()

dict_keys(['__header__', '__version__', '__globals__', 'reco'])

In [28]:
coord['reco'].keys()

dict_keys(['props', 'native', 'scrf', 'mni'])

In [31]:
coord['reco']['mni'].keys()

dict_keys(['coords_mm', 'markers', 'trajectory'])

In [12]:
coord['reco']['mni']['coords_mm']

array([array([[ 12.6147008 , -14.2120616 , -11.28948732],
       [ 12.49853076, -12.66159604,  -9.23272892],
       [ 14.25388929, -13.25507936,  -9.30083084],
       [ 13.49528669, -12.17068939, -10.63704182],
       [ 13.28393795, -11.15922788,  -7.66271177],
       [ 15.02730217, -11.74798844,  -7.73018084],
       [ 14.28660453, -10.67063467,  -9.05097516],
       [ 14.95862434,  -9.68565816,  -6.55095422]]),
       array([[-11.78761068, -15.2654857 , -10.60924622],
       [-13.35010537, -13.88430303,  -9.54170327],
       [-12.16813592, -14.23507876,  -8.21780558],
       [-11.66375289, -13.17528313,  -9.70767162],
       [-13.94109811, -12.39308285,  -8.08068538],
       [-12.75964271, -12.73955877,  -6.76548167],
       [-12.25290039, -11.69044692,  -8.24852457],
       [-13.55723045, -10.77361787,  -6.22161075]])], dtype=object)

In [158]:
ind = 0
df = dfs[ind]
ch_names_r = ["LFP_R_1_STN_BS",
            "LFP_R_2_STN_BS",
            "LFP_R_3_STN_BS",
            "LFP_R_4_STN_BS",
            "LFP_R_5_STN_BS",
            "LFP_R_6_STN_BS",
            "LFP_R_7_STN_BS",
            "LFP_R_8_STN_BS"]
ch_names_l = ["LFP_L_1_STN_BS",
            "LFP_L_2_STN_BS",
            "LFP_L_3_STN_BS",
            "LFP_L_4_STN_BS",
            "LFP_L_5_STN_BS",
            "LFP_L_6_STN_BS",
            "LFP_L_7_STN_BS",
            "LFP_L_8_STN_BS"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-158-2fc01361c977>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [159]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [160]:
#model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [161]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_2_SMC_,-46.000000,-15.500000,63.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_1_3_SMC_,-45.500000,-5.500000,60.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_1_4_SMC_,-44.500000,4.000000,57.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_1_5_SMC_,-42.000000,14.000000,53.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_1_6_SMC_,-39.000000,22.500000,49.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_BS,12.614701,-14.212062,-11.289487,6.0,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_BS,12.498531,-12.661596,-9.232729,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_BS,14.253889,-13.255079,-9.300831,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_4_STN_BS,13.495287,-12.170689,-10.637042,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]


In [162]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [163]:
ind = 1
print(electrodes[ind])
df = dfs[ind]
ch_names_r = ["LFP_R_1_STN_BS",
            "LFP_R_2_STN_BS",
            "LFP_R_3_STN_BS",
            "LFP_R_4_STN_BS",
            "LFP_R_5_STN_BS",
            "LFP_R_6_STN_BS",
            "LFP_R_7_STN_BS",
            "LFP_R_8_STN_BS"]
ch_names_l = ["LFP_L_1_STN_BS",
            "LFP_L_2_STN_BS",
            "LFP_L_3_STN_BS",
            "LFP_L_4_STN_BS",
            "LFP_L_5_STN_BS",
            "LFP_L_6_STN_BS",
            "LFP_L_7_STN_BS",
            "LFP_L_8_STN_BS"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-001\ses-EphysMedOn01\ieeg\sub-001_ses-EphysMedOn01_space-MNI152NLin2009bAsym_electrodes.tsv


<ipython-input-163-1ad5de070571>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [165]:
df = pd.read_csv(electrodes[1], sep='\t', index_col=0)

In [166]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-47.000000,-26.000000,65.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_2_SMC_AT,-46.000000,-15.500000,63.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_3_SMC_AT,-45.500000,-5.500000,60.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_4_SMC_AT,-44.500000,4.000000,57.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_5_SMC_AT,-42.000000,14.000000,53.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_6_SMC_AT,-39.000000,22.500000,49.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
LFP_R_1_STN_BS,12.614701,-14.212062,-11.289487,6.00,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_2_STN_BS,12.498531,-12.661596,-9.232729,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_3_STN_BS,14.253889,-13.255079,-9.300831,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]


In [167]:
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [168]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-47.000000,-26.000000,65.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_2_SMC_AT,-46.000000,-15.500000,63.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_3_SMC_AT,-45.500000,-5.500000,60.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_4_SMC_AT,-44.500000,4.000000,57.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_5_SMC_AT,-42.000000,14.000000,53.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_6_SMC_AT,-39.000000,22.500000,49.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
LFP_R_1_STN_BS,12.614701,-14.212062,-11.289487,6.00,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_2_STN_BS,12.498531,-12.661596,-9.232729,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_3_STN_BS,14.253889,-13.255079,-9.300831,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]


In [160]:
#model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [161]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_2_SMC_,-46.000000,-15.500000,63.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_1_3_SMC_,-45.500000,-5.500000,60.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_1_4_SMC_,-44.500000,4.000000,57.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_1_5_SMC_,-42.000000,14.000000,53.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_1_6_SMC_,-39.000000,22.500000,49.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_BS,12.614701,-14.212062,-11.289487,6.0,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_BS,12.498531,-12.661596,-9.232729,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_BS,14.253889,-13.255079,-9.300831,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_4_STN_BS,13.495287,-12.170689,-10.637042,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]


In [162]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [170]:
dfs[2]

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-47.000000,-26.000000,65.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_2_SMC_AT,-46.000000,-15.500000,63.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_3_SMC_AT,-45.500000,-5.500000,60.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_4_SMC_AT,-44.500000,4.000000,57.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_5_SMC_AT,-42.000000,14.000000,53.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_6_SMC_AT,-39.000000,22.500000,49.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
LFP_R_1_STN_BS,12.614701,-14.212062,-11.289487,6.00,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_2_STN_BS,12.498531,-12.661596,-9.232729,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_3_STN_BS,14.253889,-13.255079,-9.300831,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]


# sub-002

In [17]:
coord = coords[1]

In [115]:
coord.keys()

dict_keys(['__header__', '__version__', '__globals__', 'reco'])

In [116]:
coord['reco'].keys()

dict_keys(['props', 'native', 'scrf', 'mni'])

In [117]:
coord['reco']['mni'].keys()

dict_keys(['coords_mm', 'markers', 'trajectory'])

In [18]:
coord['reco']['mni']['coords_mm']

array([array([[ 11.8327 , -14.8891 ,  -9.10934],
       [ 12.3707 , -13.6485 ,  -7.03789],
       [ 12.0276 , -14.668  ,  -6.40611],
       [ 13.1289 , -14.5918 ,  -7.01071],
       [ 13.0713 , -13.0653 ,  -4.77084],
       [ 12.7332 , -14.108  ,  -4.15542],
       [ 13.8241 , -14.0304 ,  -4.75592],
       [ 13.9119 , -13.1935 ,  -2.28796]]),
       array([[-12.3325 , -17.1655 ,  -8.7456 ],
       [-13.0522 , -16.004  ,  -6.91854],
       [-13.5455 , -17.1799 ,  -6.53316],
       [-12.3516 , -16.6752 ,  -6.18122],
       [-13.7334 , -15.4365 ,  -4.72797],
       [-14.2192 , -16.6367 ,  -4.34364],
       [-13.0314 , -16.1272 ,  -3.98196],
       [-14.3399 , -15.5135 ,  -2.12905]])], dtype=object)

In [196]:
ind = 3
df = dfs[ind]
print(electrodes[ind])
ch_names_r = ["LFP_R_1_STN_BS",
            "LFP_R_2_STN_BS",
            "LFP_R_3_STN_BS",
            "LFP_R_4_STN_BS",
            "LFP_R_5_STN_BS",
            "LFP_R_6_STN_BS",
            "LFP_R_7_STN_BS",
            "LFP_R_8_STN_BS"]
ch_names_l = ["LFP_L_1_STN_BS",
            "LFP_L_2_STN_BS",
            "LFP_L_3_STN_BS",
            "LFP_L_4_STN_BS",
            "LFP_L_5_STN_BS",
            "LFP_L_6_STN_BS",
            "LFP_L_7_STN_BS",
            "LFP_L_8_STN_BS"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-002\ses-EphysMedOff01\ieeg\sub-002_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv


<ipython-input-196-064a3b9c5c53>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [197]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-39.5,-74.0,40.5,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_2_SMC_AT,-41.4,-65.0,46.0,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_3_SMC_AT,-41.5,-54.5,52.0,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_4_SMC_AT,-41.5,-44.5,56.0,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_5_SMC_AT,-40.5,-34.0,59.5,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_6_SMC_AT,-40.0,-24.0,60.0,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
LFP_R_1_STN_BS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LFP_R_2_STN_BS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LFP_R_3_STN_BS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [199]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-39.5000,-74.0000,40.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_2_SMC_AT,-41.4000,-65.0000,46.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_3_SMC_AT,-41.5000,-54.5000,52.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_4_SMC_AT,-41.5000,-44.5000,56.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_5_SMC_AT,-40.5000,-34.0000,59.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_6_SMC_AT,-40.0000,-24.0000,60.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
LFP_R_1_STN_BS,11.8327,-14.8891,-9.10934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LFP_R_2_STN_BS,12.3707,-13.6485,-7.03789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LFP_R_3_STN_BS,12.0276,-14.6680,-6.40611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
#model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [201]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-39.5000,-74.0000,40.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-41.4000,-65.0000,46.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-41.5000,-54.5000,52.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-41.5000,-44.5000,56.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-40.5000,-34.0000,59.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-40.0000,-24.0000,60.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_BS,11.8327,-14.8891,-9.10934,6.00,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_BS,12.3707,-13.6485,-7.03789,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_BS,12.0276,-14.6680,-6.40611,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]


In [202]:
electrodes[ind]

'C:\\Users\\richa\\OneDrive - Charité - Universitätsmedizin Berlin\\Data\\BIDS_Berlin_ECOG_LFP\\rawdata\\sub-002\\ses-EphysMedOff01\\ieeg\\sub-002_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv'

In [203]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [217]:
ind = 4
print(electrodes[ind])
df = pd.read_csv(electrodes[ind], sep='\t', index_col=0)
ch_names_r = ["LFP_R_1_STN_BS",
            "LFP_R_2_STN_BS",
            "LFP_R_3_STN_BS",
            "LFP_R_4_STN_BS",
            "LFP_R_5_STN_BS",
            "LFP_R_6_STN_BS",
            "LFP_R_7_STN_BS",
            "LFP_R_8_STN_BS"]
ch_names_l = ["LFP_L_1_STN_BS",
            "LFP_L_2_STN_BS",
            "LFP_L_3_STN_BS",
            "LFP_L_4_STN_BS",
            "LFP_L_5_STN_BS",
            "LFP_L_6_STN_BS",
            "LFP_L_7_STN_BS",
            "LFP_L_8_STN_BS"]
#for ch in ch_names_r + ch_names_l:
    #df = df.append(pd.Series(name=ch))

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-002\ses-EphysMedOff02\ieeg\sub-002_ses-EphysMedOff02_space-MNI152NLin2009bAsym_electrodes.tsv


In [218]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-39.5000,-74.00000,40.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_2_SMC_AT,-41.4000,-65.00000,46.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_3_SMC_AT,-41.5000,-54.50000,52.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_4_SMC_AT,-41.5000,-44.50000,56.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_5_SMC_AT,-40.5000,-34.00000,59.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_6_SMC_AT,-40.0000,-24.00000,60.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
LFP_R_1_STN_BS,11.0966,-13.17370,-6.813580,6.00,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_2_STN_BS,11.6184,-11.44320,-5.169890,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_3_STN_BS,11.1270,-12.27710,-4.557230,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]


In [ ]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None

In [222]:
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [223]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-39.5000,-74.0000,40.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_2_SMC_AT,-41.4000,-65.0000,46.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_3_SMC_AT,-41.5000,-54.5000,52.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_4_SMC_AT,-41.5000,-44.5000,56.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_5_SMC_AT,-40.5000,-34.0000,59.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
ECOG_L_6_SMC_AT,-40.0000,-24.0000,60.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,NaN,[1x6]
LFP_R_1_STN_BS,11.8327,-14.8891,-9.10934,6.00,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_2_STN_BS,12.3707,-13.6485,-7.03789,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]
LFP_R_3_STN_BS,12.0276,-14.6680,-6.40611,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,NaN,[1x8]


In [207]:
#model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [208]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-39.5000,-74.00000,40.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-41.4000,-65.00000,46.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-41.5000,-54.50000,52.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-41.5000,-44.50000,56.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-40.5000,-34.00000,59.500000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-40.0000,-24.00000,60.000000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_BS,11.0966,-13.17370,-6.813580,6.00,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_BS,11.6184,-11.44320,-5.169890,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_BS,11.1270,-12.27710,-4.557230,1.50,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]


In [224]:
electrodes[ind]

'C:\\Users\\richa\\OneDrive - Charité - Universitätsmedizin Berlin\\Data\\BIDS_Berlin_ECOG_LFP\\rawdata\\sub-002\\ses-EphysMedOff02\\ieeg\\sub-002_ses-EphysMedOff02_space-MNI152NLin2009bAsym_electrodes.tsv'

In [225]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [226]:
ind = 5
print(electrodes[ind])

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-002\ses-EphysMedOff03\ieeg\sub-002_ses-EphysMedOff03_space-MNI152NLin2009bAsym_electrodes.tsv


In [227]:
df = dfs[ind]
ch_names_r = ["LFP_R_1_STN_BS",
            "LFP_R_2_STN_BS",
            "LFP_R_3_STN_BS",
            "LFP_R_4_STN_BS",
            "LFP_R_5_STN_BS",
            "LFP_R_6_STN_BS",
            "LFP_R_7_STN_BS",
            "LFP_R_8_STN_BS"]
ch_names_l = ["LFP_L_1_STN_BS",
            "LFP_L_2_STN_BS",
            "LFP_L_3_STN_BS",
            "LFP_L_4_STN_BS",
            "LFP_L_5_STN_BS",
            "LFP_L_6_STN_BS",
            "LFP_L_7_STN_BS",
            "LFP_L_8_STN_BS"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-227-fc914f40bce4>:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [228]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [229]:
#model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [230]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-39.5000,-74.0000,40.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-41.4000,-65.0000,46.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-41.5000,-54.5000,52.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-41.5000,-44.5000,56.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-40.5000,-34.0000,59.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-40.0000,-24.0000,60.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_BS,11.8327,-14.8891,-9.10934,6.0,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_BS,12.3707,-13.6485,-7.03789,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_BS,12.0276,-14.6680,-6.40611,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]


In [231]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

# sub-003

In [23]:
coord = coords[2]

In [26]:
files[2]

'C:\\Users\\richa\\OneDrive - Charité - Universitätsmedizin Berlin\\Data\\BIDS_Berlin_ECOG_LFP\\sourcedata\\sub-003\\Neuroimaging\\ea_reconstruction.mat'

In [27]:
ind = 6
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-003\ses-EphysMedOff01\ieeg\sub-003_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv


In [29]:
coord['reco']['mni']['coords_mm']

array([array([[ 12.3329 , -14.1452 ,  -9.19623],
       [ 13.2073 , -13.2389 ,  -7.32322],
       [ 14.0552 , -12.3317 ,  -5.43968],
       [ 14.8684 , -11.4021 ,  -3.54888]]),
       array([[-10.4793 , -15.6278 ,  -8.22072],
       [-11.157  , -14.5562 ,  -6.39784],
       [-11.798  , -13.483  ,  -4.56256],
       [-12.4066 , -12.3907 ,  -2.70754]])], dtype=object)

In [30]:
ch_names_r = ["LFP_R_1_STN_MT",
            "LFP_R_2_STN_MT",
            "LFP_R_3_STN_MT",
            "LFP_R_4_STN_MT"]
ch_names_l = ["LFP_L_1_STN_MT",
            "LFP_L_2_STN_MT",
            "LFP_L_3_STN_MT",
            "LFP_L_4_STN_MT"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-30-0baf73d9ddaa>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [31]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [32]:
#model = "SenSight"
model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [33]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-23.5000,-53.0000,72.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-25.5000,-42.5000,72.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-27.5000,-32.0000,72.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-28.0000,-22.0000,71.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-28.0000,-10.5000,68.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-28.0000,0.5000,63.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_MT,12.3329,-14.1452,-9.19623,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]
LFP_R_2_STN_MT,13.2073,-13.2389,-7.32322,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]
LFP_R_3_STN_MT,14.0552,-12.3317,-5.43968,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]


In [34]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [35]:
ind = 7
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-003\ses-EphysMedOn01\ieeg\sub-003_ses-EphysMedOn01_space-MNI152NLin2009bAsym_electrodes.tsv


In [36]:
coord['reco']['mni']['coords_mm']

array([array([[ 12.3329 , -14.1452 ,  -9.19623],
       [ 13.2073 , -13.2389 ,  -7.32322],
       [ 14.0552 , -12.3317 ,  -5.43968],
       [ 14.8684 , -11.4021 ,  -3.54888]]),
       array([[-10.4793 , -15.6278 ,  -8.22072],
       [-11.157  , -14.5562 ,  -6.39784],
       [-11.798  , -13.483  ,  -4.56256],
       [-12.4066 , -12.3907 ,  -2.70754]])], dtype=object)

In [37]:
ch_names_r = ["LFP_R_1_STN_MT",
            "LFP_R_2_STN_MT",
            "LFP_R_3_STN_MT",
            "LFP_R_4_STN_MT"]
ch_names_l = ["LFP_L_1_STN_MT",
            "LFP_L_2_STN_MT",
            "LFP_L_3_STN_MT",
            "LFP_L_4_STN_MT"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-37-0baf73d9ddaa>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [38]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [39]:
#model = "SenSight"
model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [40]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-23.5000,-53.0000,72.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-25.5000,-42.5000,72.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-27.5000,-32.0000,72.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-28.0000,-22.0000,71.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-28.0000,-10.5000,68.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-28.0000,0.5000,63.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_MT,12.3329,-14.1452,-9.19623,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]
LFP_R_2_STN_MT,13.2073,-13.2389,-7.32322,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]
LFP_R_3_STN_MT,14.0552,-12.3317,-5.43968,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]


In [41]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [42]:
ind = 8
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-003\ses-EphysMedOn02\ieeg\sub-003_ses-EphysMedOn02_space-MNI152NLin2009bAsym_electrodes.tsv


In [43]:
coord['reco']['mni']['coords_mm']

array([array([[ 12.3329 , -14.1452 ,  -9.19623],
       [ 13.2073 , -13.2389 ,  -7.32322],
       [ 14.0552 , -12.3317 ,  -5.43968],
       [ 14.8684 , -11.4021 ,  -3.54888]]),
       array([[-10.4793 , -15.6278 ,  -8.22072],
       [-11.157  , -14.5562 ,  -6.39784],
       [-11.798  , -13.483  ,  -4.56256],
       [-12.4066 , -12.3907 ,  -2.70754]])], dtype=object)

In [44]:
ch_names_r = ["LFP_R_1_STN_MT",
            "LFP_R_2_STN_MT",
            "LFP_R_3_STN_MT",
            "LFP_R_4_STN_MT"]
ch_names_l = ["LFP_L_1_STN_MT",
            "LFP_L_2_STN_MT",
            "LFP_L_3_STN_MT",
            "LFP_L_4_STN_MT"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-44-0baf73d9ddaa>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [45]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [46]:
#model = "SenSight"
model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [47]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-23.5000,-53.0000,72.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-25.5000,-42.5000,72.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-27.5000,-32.0000,72.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-28.0000,-22.0000,71.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-28.0000,-10.5000,68.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-28.0000,0.5000,63.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_MT,12.3329,-14.1452,-9.19623,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]
LFP_R_2_STN_MT,13.2073,-13.2389,-7.32322,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]
LFP_R_3_STN_MT,14.0552,-12.3317,-5.43968,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]


In [48]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [49]:
ind = 9
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-003\ses-EphysMedOn03\ieeg\sub-003_ses-EphysMedOn03_space-MNI152NLin2009bAsym_electrodes.tsv


In [50]:
coord['reco']['mni']['coords_mm']

array([array([[ 12.3329 , -14.1452 ,  -9.19623],
       [ 13.2073 , -13.2389 ,  -7.32322],
       [ 14.0552 , -12.3317 ,  -5.43968],
       [ 14.8684 , -11.4021 ,  -3.54888]]),
       array([[-10.4793 , -15.6278 ,  -8.22072],
       [-11.157  , -14.5562 ,  -6.39784],
       [-11.798  , -13.483  ,  -4.56256],
       [-12.4066 , -12.3907 ,  -2.70754]])], dtype=object)

In [51]:
ch_names_r = ["LFP_R_1_STN_MT",
            "LFP_R_2_STN_MT",
            "LFP_R_3_STN_MT",
            "LFP_R_4_STN_MT"]
ch_names_l = ["LFP_L_1_STN_MT",
            "LFP_L_2_STN_MT",
            "LFP_L_3_STN_MT",
            "LFP_L_4_STN_MT"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-51-0baf73d9ddaa>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [52]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [53]:
#model = "SenSight"
model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [54]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-23.5000,-53.0000,72.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-25.5000,-42.5000,72.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-27.5000,-32.0000,72.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-28.0000,-22.0000,71.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-28.0000,-10.5000,68.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-28.0000,0.5000,63.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_MT,12.3329,-14.1452,-9.19623,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]
LFP_R_2_STN_MT,13.2073,-13.2389,-7.32322,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]
LFP_R_3_STN_MT,14.0552,-12.3317,-5.43968,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x4]


In [55]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

# sub-004

In [57]:
coord = coords[3]

In [56]:
ind = 10
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-004\ses-EphysMedOff01\ieeg\sub-004_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv


In [58]:
coord['reco']['mni']['coords_mm']

array([array([[ 11.1667 , -14.8884 , -11.2823 ],
       [ 11.2784 , -14.1758 ,  -8.75138],
       [ 12.503  , -14.46   ,  -8.96336],
       [ 11.8716 , -13.3525 ,  -9.51646],
       [ 11.9986 , -13.2779 ,  -6.59159],
       [ 13.2167 , -13.5698 ,  -6.79572],
       [ 12.5889 , -12.4489 ,  -7.34254],
       [ 13.3072 , -12.2019 ,  -4.77584]]),
       array([[-12.6911 , -15.0769 , -10.0615 ],
       [-13.9828 , -14.0101 ,  -8.25431],
       [-13.418  , -14.8026 ,  -7.44341],
       [-12.7443 , -13.7049 ,  -8.03359],
       [-14.6675 , -13.1042 ,  -6.14035],
       [-14.1087 , -13.9054 ,  -5.34159],
       [-13.4307 , -12.7885 ,  -5.92301],
       [-14.747  , -12.3596 ,  -3.72426]])], dtype=object)

In [59]:
ch_names_r = ["LFP_R_1_STN_BS",
            "LFP_R_2_STN_BS",
            "LFP_R_3_STN_BS",
            "LFP_R_4_STN_BS",
            "LFP_R_5_STN_BS",
            "LFP_R_6_STN_BS",
            "LFP_R_7_STN_BS",
            "LFP_R_8_STN_BS"]
ch_names_l = ["LFP_L_1_STN_BS",
            "LFP_L_2_STN_BS",
            "LFP_L_3_STN_BS",
            "LFP_L_4_STN_BS",
            "LFP_L_5_STN_BS",
            "LFP_L_6_STN_BS",
            "LFP_L_7_STN_BS",
            "LFP_L_8_STN_BS"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-59-4079a66a5386>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [60]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [61]:
#model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [62]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-14.5000,-68.5000,67.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-19.5000,-59.5000,72.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-24.0000,-50.5000,74.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-26.5000,-40.0000,74.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-27.0000,-30.0000,73.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-27.0000,-18.5000,70.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_BS,11.1667,-14.8884,-11.28230,6.0,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_BS,11.2784,-14.1758,-8.75138,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_BS,12.5030,-14.4600,-8.96336,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]


In [63]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [64]:
ind = 11
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-004\ses-EphysMedOn01\ieeg\sub-004_ses-EphysMedOn01_space-MNI152NLin2009bAsym_electrodes.tsv


In [65]:
coord['reco']['mni']['coords_mm']

array([array([[ 11.1667 , -14.8884 , -11.2823 ],
       [ 11.2784 , -14.1758 ,  -8.75138],
       [ 12.503  , -14.46   ,  -8.96336],
       [ 11.8716 , -13.3525 ,  -9.51646],
       [ 11.9986 , -13.2779 ,  -6.59159],
       [ 13.2167 , -13.5698 ,  -6.79572],
       [ 12.5889 , -12.4489 ,  -7.34254],
       [ 13.3072 , -12.2019 ,  -4.77584]]),
       array([[-12.6911 , -15.0769 , -10.0615 ],
       [-13.9828 , -14.0101 ,  -8.25431],
       [-13.418  , -14.8026 ,  -7.44341],
       [-12.7443 , -13.7049 ,  -8.03359],
       [-14.6675 , -13.1042 ,  -6.14035],
       [-14.1087 , -13.9054 ,  -5.34159],
       [-13.4307 , -12.7885 ,  -5.92301],
       [-14.747  , -12.3596 ,  -3.72426]])], dtype=object)

In [66]:
ch_names_r = ["LFP_R_1_STN_BS",
            "LFP_R_2_STN_BS",
            "LFP_R_3_STN_BS",
            "LFP_R_4_STN_BS",
            "LFP_R_5_STN_BS",
            "LFP_R_6_STN_BS",
            "LFP_R_7_STN_BS",
            "LFP_R_8_STN_BS"]
ch_names_l = ["LFP_L_1_STN_BS",
            "LFP_L_2_STN_BS",
            "LFP_L_3_STN_BS",
            "LFP_L_4_STN_BS",
            "LFP_L_5_STN_BS",
            "LFP_L_6_STN_BS",
            "LFP_L_7_STN_BS",
            "LFP_L_8_STN_BS"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-66-4079a66a5386>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [67]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [68]:
#model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [69]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-14.5000,-68.5000,67.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-19.5000,-59.5000,72.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-24.0000,-50.5000,74.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-26.5000,-40.0000,74.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-27.0000,-30.0000,73.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-27.0000,-18.5000,70.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_BS,11.1667,-14.8884,-11.28230,6.0,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_BS,11.2784,-14.1758,-8.75138,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_BS,12.5030,-14.4600,-8.96336,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]


In [70]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [71]:
ind = 12
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-004\ses-EphysMedOn02\ieeg\sub-004_ses-EphysMedOn02_space-MNI152NLin2009bAsym_electrodes.tsv


In [72]:
coord['reco']['mni']['coords_mm']

array([array([[ 11.1667 , -14.8884 , -11.2823 ],
       [ 11.2784 , -14.1758 ,  -8.75138],
       [ 12.503  , -14.46   ,  -8.96336],
       [ 11.8716 , -13.3525 ,  -9.51646],
       [ 11.9986 , -13.2779 ,  -6.59159],
       [ 13.2167 , -13.5698 ,  -6.79572],
       [ 12.5889 , -12.4489 ,  -7.34254],
       [ 13.3072 , -12.2019 ,  -4.77584]]),
       array([[-12.6911 , -15.0769 , -10.0615 ],
       [-13.9828 , -14.0101 ,  -8.25431],
       [-13.418  , -14.8026 ,  -7.44341],
       [-12.7443 , -13.7049 ,  -8.03359],
       [-14.6675 , -13.1042 ,  -6.14035],
       [-14.1087 , -13.9054 ,  -5.34159],
       [-13.4307 , -12.7885 ,  -5.92301],
       [-14.747  , -12.3596 ,  -3.72426]])], dtype=object)

In [73]:
ch_names_r = ["LFP_R_1_STN_BS",
            "LFP_R_2_STN_BS",
            "LFP_R_3_STN_BS",
            "LFP_R_4_STN_BS",
            "LFP_R_5_STN_BS",
            "LFP_R_6_STN_BS",
            "LFP_R_7_STN_BS",
            "LFP_R_8_STN_BS"]
ch_names_l = ["LFP_L_1_STN_BS",
            "LFP_L_2_STN_BS",
            "LFP_L_3_STN_BS",
            "LFP_L_4_STN_BS",
            "LFP_L_5_STN_BS",
            "LFP_L_6_STN_BS",
            "LFP_L_7_STN_BS",
            "LFP_L_8_STN_BS"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-73-4079a66a5386>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [74]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [75]:
#model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [76]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_L_1_SMC_AT,-14.5000,-68.5000,67.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_2_SMC_AT,-19.5000,-59.5000,72.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_3_SMC_AT,-24.0000,-50.5000,74.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_4_SMC_AT,-26.5000,-40.0000,74.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_5_SMC_AT,-27.0000,-30.0000,73.00000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
ECOG_L_6_SMC_AT,-27.0000,-18.5000,70.50000,12.57,platinum,Ad-Tech,ECOG_strip,L,strip,n/a,[1x6]
LFP_R_1_STN_BS,11.1667,-14.8884,-11.28230,6.0,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_BS,11.2784,-14.1758,-8.75138,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_BS,12.5030,-14.4600,-8.96336,1.5,platinum/iridium,Boston Scientific,DBS_right,R,depth,n/a,[1x8]


In [77]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

# sub-005

In [79]:
ind = 13
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-005\ses-EphysMedOff01\ieeg\sub-005_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv


In [80]:
coord = coords[4]

In [81]:
coord['reco']['mni']['coords_mm']

array([array([[ 11.0966  , -13.1737  ,  -6.81358 ],
       [ 11.6184  , -11.4432  ,  -5.16989 ],
       [ 11.127   , -12.2771  ,  -4.55723 ],
       [ 12.3174  , -12.4006  ,  -4.78961 ],
       [ 12.258   , -10.2834  ,  -3.22074 ],
       [ 11.7614  , -11.1216  ,  -2.5914  ],
       [ 12.9546  , -11.2482  ,  -2.83966 ],
       [ 12.9948  ,  -9.66488 ,  -0.828834]]),
       array([[-10.9348 , -14.9467 ,  -8.05791],
       [-11.5714 , -13.3614 ,  -6.35134],
       [-12.3478 , -14.3259 ,  -6.09908],
       [-11.2299 , -14.3398 ,  -5.7675 ],
       [-12.4289 , -12.4315 ,  -4.39711],
       [-13.2019 , -13.4017 ,  -4.15436],
       [-12.0805 , -13.4141 ,  -3.79927],
       [-13.4946 , -12.1292 ,  -2.07095]])], dtype=object)

In [82]:
ch_names_r = ["LFP_R_1_STN_MT",
            "LFP_R_2_STN_MT",
            "LFP_R_3_STN_MT",
            "LFP_R_4_STN_MT",
            "LFP_R_5_STN_MT",
            "LFP_R_6_STN_MT",
            "LFP_R_7_STN_MT",
            "LFP_R_8_STN_MT"]
ch_names_l = ["LFP_L_1_STN_MT",
            "LFP_L_2_STN_MT",
            "LFP_L_3_STN_MT",
            "LFP_L_4_STN_MT",
            "LFP_L_5_STN_MT",
            "LFP_L_6_STN_MT",
            "LFP_L_7_STN_MT",
            "LFP_L_8_STN_MT"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-82-1a9ffdbb173c>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [83]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [84]:
model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [85]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_R_1_SMC_AT,18.0000,-70.50000,65.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_2_SMC_AT,22.0000,-60.50000,70.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_3_SMC_AT,35.0000,-50.50000,73.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_4_SMC_AT,27.5000,-39.60000,73.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_5_SMC_AT,28.0000,-27.00000,72.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_6_SMC_AT,28.0000,-17.50000,71.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
LFP_R_1_STN_MT,11.0966,-13.17370,-6.813580,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_MT,11.6184,-11.44320,-5.169890,1.5,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_MT,11.1270,-12.27710,-4.557230,1.5,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]


In [86]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [87]:
ind = 14
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-005\ses-EphysMedOff02\ieeg\sub-005_ses-EphysMedOff02_space-MNI152NLin2009bAsym_electrodes.tsv


In [88]:
coord['reco']['mni']['coords_mm']

array([array([[ 11.0966  , -13.1737  ,  -6.81358 ],
       [ 11.6184  , -11.4432  ,  -5.16989 ],
       [ 11.127   , -12.2771  ,  -4.55723 ],
       [ 12.3174  , -12.4006  ,  -4.78961 ],
       [ 12.258   , -10.2834  ,  -3.22074 ],
       [ 11.7614  , -11.1216  ,  -2.5914  ],
       [ 12.9546  , -11.2482  ,  -2.83966 ],
       [ 12.9948  ,  -9.66488 ,  -0.828834]]),
       array([[-10.9348 , -14.9467 ,  -8.05791],
       [-11.5714 , -13.3614 ,  -6.35134],
       [-12.3478 , -14.3259 ,  -6.09908],
       [-11.2299 , -14.3398 ,  -5.7675 ],
       [-12.4289 , -12.4315 ,  -4.39711],
       [-13.2019 , -13.4017 ,  -4.15436],
       [-12.0805 , -13.4141 ,  -3.79927],
       [-13.4946 , -12.1292 ,  -2.07095]])], dtype=object)

In [89]:
ch_names_r = ["LFP_R_1_STN_MT",
            "LFP_R_2_STN_MT",
            "LFP_R_3_STN_MT",
            "LFP_R_4_STN_MT",
            "LFP_R_5_STN_MT",
            "LFP_R_6_STN_MT",
            "LFP_R_7_STN_MT",
            "LFP_R_8_STN_MT"]
ch_names_l = ["LFP_L_1_STN_MT",
            "LFP_L_2_STN_MT",
            "LFP_L_3_STN_MT",
            "LFP_L_4_STN_MT",
            "LFP_L_5_STN_MT",
            "LFP_L_6_STN_MT",
            "LFP_L_7_STN_MT",
            "LFP_L_8_STN_MT"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-89-1a9ffdbb173c>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [90]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [91]:
model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [92]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_R_1_SMC_AT,18.0000,-70.50000,65.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_2_SMC_AT,22.0000,-60.50000,70.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_3_SMC_AT,35.0000,-50.50000,73.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_4_SMC_AT,27.5000,-39.60000,73.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_5_SMC_AT,28.0000,-27.00000,72.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_6_SMC_AT,28.0000,-17.50000,71.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
LFP_R_1_STN_MT,11.0966,-13.17370,-6.813580,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_MT,11.6184,-11.44320,-5.169890,1.5,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_MT,11.1270,-12.27710,-4.557230,1.5,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]


In [93]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [94]:
ind = 15
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-005\ses-EphysMedOn01\ieeg\sub-005_ses-EphysMedOn01_space-MNI152NLin2009bAsym_electrodes.tsv


In [95]:
coord['reco']['mni']['coords_mm']

array([array([[ 11.0966  , -13.1737  ,  -6.81358 ],
       [ 11.6184  , -11.4432  ,  -5.16989 ],
       [ 11.127   , -12.2771  ,  -4.55723 ],
       [ 12.3174  , -12.4006  ,  -4.78961 ],
       [ 12.258   , -10.2834  ,  -3.22074 ],
       [ 11.7614  , -11.1216  ,  -2.5914  ],
       [ 12.9546  , -11.2482  ,  -2.83966 ],
       [ 12.9948  ,  -9.66488 ,  -0.828834]]),
       array([[-10.9348 , -14.9467 ,  -8.05791],
       [-11.5714 , -13.3614 ,  -6.35134],
       [-12.3478 , -14.3259 ,  -6.09908],
       [-11.2299 , -14.3398 ,  -5.7675 ],
       [-12.4289 , -12.4315 ,  -4.39711],
       [-13.2019 , -13.4017 ,  -4.15436],
       [-12.0805 , -13.4141 ,  -3.79927],
       [-13.4946 , -12.1292 ,  -2.07095]])], dtype=object)

In [96]:
ch_names_r = ["LFP_R_1_STN_MT",
            "LFP_R_2_STN_MT",
            "LFP_R_3_STN_MT",
            "LFP_R_4_STN_MT",
            "LFP_R_5_STN_MT",
            "LFP_R_6_STN_MT",
            "LFP_R_7_STN_MT",
            "LFP_R_8_STN_MT"]
ch_names_l = ["LFP_L_1_STN_MT",
            "LFP_L_2_STN_MT",
            "LFP_L_3_STN_MT",
            "LFP_L_4_STN_MT",
            "LFP_L_5_STN_MT",
            "LFP_L_6_STN_MT",
            "LFP_L_7_STN_MT",
            "LFP_L_8_STN_MT"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-96-1a9ffdbb173c>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [97]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [98]:
model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [99]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_R_1_SMC_AT,18.0000,-70.50000,65.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_2_SMC_AT,22.0000,-60.50000,70.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_3_SMC_AT,35.0000,-50.50000,73.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_4_SMC_AT,27.5000,-39.60000,73.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_5_SMC_AT,28.0000,-27.00000,72.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_6_SMC_AT,28.0000,-17.50000,71.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
LFP_R_1_STN_MT,11.0966,-13.17370,-6.813580,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_MT,11.6184,-11.44320,-5.169890,1.5,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_MT,11.1270,-12.27710,-4.557230,1.5,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]


In [100]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

In [101]:
ind = 16
print(electrodes[ind])
df = dfs[ind]

C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-005\ses-EphysMedOn02\ieeg\sub-005_ses-EphysMedOn02_space-MNI152NLin2009bAsym_electrodes.tsv


In [102]:
coord['reco']['mni']['coords_mm']

array([array([[ 11.0966  , -13.1737  ,  -6.81358 ],
       [ 11.6184  , -11.4432  ,  -5.16989 ],
       [ 11.127   , -12.2771  ,  -4.55723 ],
       [ 12.3174  , -12.4006  ,  -4.78961 ],
       [ 12.258   , -10.2834  ,  -3.22074 ],
       [ 11.7614  , -11.1216  ,  -2.5914  ],
       [ 12.9546  , -11.2482  ,  -2.83966 ],
       [ 12.9948  ,  -9.66488 ,  -0.828834]]),
       array([[-10.9348 , -14.9467 ,  -8.05791],
       [-11.5714 , -13.3614 ,  -6.35134],
       [-12.3478 , -14.3259 ,  -6.09908],
       [-11.2299 , -14.3398 ,  -5.7675 ],
       [-12.4289 , -12.4315 ,  -4.39711],
       [-13.2019 , -13.4017 ,  -4.15436],
       [-12.0805 , -13.4141 ,  -3.79927],
       [-13.4946 , -12.1292 ,  -2.07095]])], dtype=object)

In [103]:
ch_names_r = ["LFP_R_1_STN_MT",
            "LFP_R_2_STN_MT",
            "LFP_R_3_STN_MT",
            "LFP_R_4_STN_MT",
            "LFP_R_5_STN_MT",
            "LFP_R_6_STN_MT",
            "LFP_R_7_STN_MT",
            "LFP_R_8_STN_MT"]
ch_names_l = ["LFP_L_1_STN_MT",
            "LFP_L_2_STN_MT",
            "LFP_L_3_STN_MT",
            "LFP_L_4_STN_MT",
            "LFP_L_5_STN_MT",
            "LFP_L_6_STN_MT",
            "LFP_L_7_STN_MT",
            "LFP_L_8_STN_MT"]
for ch in ch_names_r + ch_names_l:
    df = df.append(pd.Series(name=ch))

<ipython-input-103-1a9ffdbb173c>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(name=ch))


In [104]:
df['material'] = None
df['manufacturer'] = None
df['group'] = None
df['hemisphere'] = None
df['type'] = None
df['impedance'] = None
df['dimension'] = None
for idx, ch_names in enumerate([ch_names_r, ch_names_l]):
    xyz = coord['reco']['mni']['coords_mm'][idx]
    for ch_n, ch in enumerate(ch_names):
        for i, ax in enumerate(['x', 'y', 'z']):
            df.loc[ch, ax] = xyz[ch_n, i]

In [105]:
model = "SenSight"
#model = 3389
for ch_name in df.index:
    if 'ECOG' in ch_name:
        df.loc[ch_name, 'material'] = "platinum"
        df.loc[ch_name, 'manufacturer'] = "Ad-Tech"
        df.loc[ch_name, 'group'] = 'ECOG_strip'
        df.loc[ch_name, 'type'] = 'strip'
        if '_L_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'L'
        elif '_R_' in ch_name:
            df.loc[ch_name, 'hemisphere'] = 'R'
        df.loc[ch_name, 'impedance'] = 'n/a'
        df.loc[ch_name, 'dimension'] = "[1x6]"
        df.loc[ch_name, 'size'] = 12.57
    elif 'LFP' in ch_name:
        df.loc[ch_name, 'material'] = "platinum/iridium"
        df.loc[ch_name, 'type'] = 'depth'
        df.loc[ch_name, 'impedance'] = 'n/a'
        if "LFP_R" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_right'
            df.loc[ch_name, 'hemisphere'] = 'R'
        elif "LFP_L" in ch_name:
            df.loc[ch_name, 'group'] = 'DBS_left'
            df.loc[ch_name, 'hemisphere'] = 'L'
        if 'MT' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Medtronic"
            if model == "SenSight":
                df.loc[ch_name, 'dimension'] = "[1x8]"
                if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
                else:
                    df.loc[ch_name, 'size'] = 1.5
            elif model == 3389:
                df.loc[ch_name, 'dimension'] = "[1x4]"
                df.loc[ch_name, 'size'] = 6.0
        elif 'BS' in ch_name:
            df.loc[ch_name, 'manufacturer'] = "Boston Scientific"
            df.loc[ch_name, 'dimension'] = "[1x8]"
            if any(["1" in ch_name, "8" in ch_name]):
                    df.loc[ch_name, 'size'] = 6.0
            else:
                df.loc[ch_name, 'size'] = 1.5

In [106]:
df

,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance,dimension
name,,,,,,,,,,,
ECOG_R_1_SMC_AT,18.0000,-70.50000,65.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_2_SMC_AT,22.0000,-60.50000,70.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_3_SMC_AT,35.0000,-50.50000,73.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_4_SMC_AT,27.5000,-39.60000,73.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_5_SMC_AT,28.0000,-27.00000,72.500000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
ECOG_R_6_SMC_AT,28.0000,-17.50000,71.000000,12.57,platinum,Ad-Tech,ECOG_strip,R,strip,n/a,[1x6]
LFP_R_1_STN_MT,11.0966,-13.17370,-6.813580,6.0,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]
LFP_R_2_STN_MT,11.6184,-11.44320,-5.169890,1.5,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]
LFP_R_3_STN_MT,11.1270,-12.27710,-4.557230,1.5,platinum/iridium,Medtronic,DBS_right,R,depth,n/a,[1x8]


In [107]:
df.to_csv(electrodes[ind], sep='\t', na_rep='n/a', index=True)

# Now updated sidecar coordsystem.json

In [109]:
root = r'C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata'
coordsystems = get_all_files(
    path=root,
    suffix='coordsystem.json',
    get_bids=True,
    prefix=None,
    bids_root=root,
    verbose=True,
    extension='json')

Corresponding files found:
0 : sub-001_ses-EphysMedOff01_coordsystem.json
1 : sub-001_ses-EphysMedOn01_coordsystem.json
2 : sub-001_ses-EphysMedOn02_coordsystem.json
3 : sub-002_ses-EphysMedOff01_coordsystem.json
4 : sub-002_ses-EphysMedOff02_coordsystem.json
5 : sub-002_ses-EphysMedOff03_coordsystem.json
6 : sub-003_ses-EphysMedOff01_coordsystem.json
7 : sub-003_ses-EphysMedOn01_coordsystem.json
8 : sub-003_ses-EphysMedOn02_coordsystem.json
9 : sub-003_ses-EphysMedOn03_coordsystem.json
10 : sub-004_ses-EphysMedOff01_coordsystem.json
11 : sub-004_ses-EphysMedOn01_coordsystem.json
12 : sub-004_ses-EphysMedOn02_coordsystem.json
13 : sub-005_ses-EphysMedOff01_coordsystem.json
14 : sub-005_ses-EphysMedOff02_coordsystem.json
15 : sub-005_ses-EphysMedOn01_coordsystem.json
16 : sub-005_ses-EphysMedOn02_coordsystem.json


In [114]:
coordsys.update(space='MNI152NLin2009bAsym')

BIDSPath(
root: C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata
datatype: ieeg
basename: sub-001_ses-EphysMedOff01_space-MNI152NLin2009bAsym_coordsystem.json)

In [115]:
for coordsys in coordsystems:
    coordsys.update(datatype='ieeg', space='MNI152NLin2009bAsym')
    entries = {
        "iEEGCoordinateSystemDescription": "MNI152 2009b NLIN asymmetric T2 template",
        "iEEGCoordinateProcessingDescription": "Co-registration, normalization and electrode localization done with Lead-DBS",
        "iEEGCoordinateProcessingReference": "Horn, A., Li, N., Dembek, T. A., Kappel, A., Boulay, C., Ewert, S., et al. (2018). Lead-DBS v2: Towards a comprehensive pipeline for deep brain stimulation imaging. NeuroImage."
    }
    mne_bids.update_sidecar_json(coordsys, entries, verbose=False)